In [10]:
import cv2
import numpy as np

# Initialize global variables
ix, iy = -1, -1
drawing = False
max_size = 50  # Maximum size of the bounding box

# Mouse callback function to draw rectangle
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img, img_name

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            # Adjust the rectangle size to not exceed the maximum size
            rect_width = min(max_size, abs(x - ix))
            rect_height = min(max_size, abs(y - iy))
            if x < ix:
                rect_x = ix - rect_width
            else:
                rect_x = ix
            if y < iy:
                rect_y = iy - rect_height
            else:
                rect_y = iy
            
            cv2.rectangle(img_copy, (ix, iy), (rect_x + rect_width, rect_y + rect_height), (0, 255, 0), 2)
            cv2.imshow(img_name, img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        rect_width = min(max_size, abs(x - ix))
        rect_height = min(max_size, abs(y - iy))
        if x < ix:
            rect_x = ix - rect_width
        else:
            rect_x = ix
        if y < iy:
            rect_y = iy - rect_height
        else:
            rect_y = iy

        # Extracting the ROI and removing the rectangle border
        roi = img[iy:rect_y + rect_height, ix:rect_x + rect_width]
        cv2.imshow(img_name, roi)
        cv2.waitKey(1)  # Display ROI for a brief moment
        save_path = f'{img_name}_patch.jpg'
        cv2.imwrite(save_path, roi)
        print(f'Patch saved to {save_path}')
        cv2.destroyAllWindows()  # Close window after saving

# Function to process each image
def process_image(image_path):
    global img, img_name
    img = cv2.imread(image_path)
    img_name = image_path.split('/')[-1].split('.')[0]  # Extract base name for window and saving
    cv2.namedWindow(img_name)
    cv2.setMouseCallback(img_name, draw_rectangle)

    while True:
        cv2.imshow(img_name, img)
        if cv2.waitKey(20) & 0xFF == 27:  # Break loop on 'ESC'
            break

    cv2.destroyAllWindows()

# Process each image
process_image('image1.jpg')
process_image('image2.jpg')


Patch saved to image1_patch.jpg
Patch saved to image2_patch.jpg


In [11]:

def compute_sift_features(image):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()
    # Find keypoints and descriptors with SIFT
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def calculate_ssd(descriptor1, descriptor2):
    # Ensure that there is at least one descriptor in each patch
    if descriptor1 is not None and descriptor2 is not None:
        # Compute the sum of squared differences (SSD) between two sets of descriptors
        ssd = np.sum((descriptor1 - descriptor2) ** 2)
        return ssd
    else:
        return None

# Load patches
patch1 = cv2.imread('image1_patch.jpg', cv2.IMREAD_GRAYSCALE)
patch2 = cv2.imread('image2_patch.jpg', cv2.IMREAD_GRAYSCALE)

# Compute SIFT features
_, descriptors1 = compute_sift_features(patch1)
_, descriptors2 = compute_sift_features(patch2)

# Calculate SSD between descriptors
if descriptors1 is not None and descriptors2 is not None:
    # Typically, you match descriptors, but here we'll compute SSD for the first descriptor for simplicity
    ssd_value = calculate_ssd(descriptors1[0], descriptors2[0])
    print("Sum of Squared Differences (SSD) between SIFT descriptors:", ssd_value)
else:
    print("Descriptors could not be computed for one or both patches.")


Sum of Squared Differences (SSD) between SIFT descriptors: 109760.0
